## Generate reactions

In [1]:
from rmgpy.species import Species, TransitionState
from rmgpy.reaction import Reaction
from rmgpy.statmech.translation import Translation, IdealGasTranslation
from rmgpy.statmech.rotation import Rotation, LinearRotor, NonlinearRotor, KRotor, SphericalTopRotor
from rmgpy.statmech.vibration import Vibration, HarmonicOscillator
from rmgpy.statmech.torsion import Torsion, HinderedRotor
from rmgpy.statmech.conformer import Conformer
from rmgpy.kinetics import Arrhenius
from rmgpy.thermo import Wilhoit
import rmgpy.constants as constants
from rmgpy.rmg.main import RMG
from rmgpy.rmg.model import CoreEdgeReactionModel
from rmgpy import settings
from IPython.display import display
from rmgpy.cantherm.output import prettify

#### Declare database variables here by changing the thermo and reaction libraries, or restrict to certain reaction families.

In [10]:
database = """
database(
    thermoLibraries = ['KlippensteinH2O2','NitrogenCurran','primaryThermoLibrary','thermo_DFT_CCSDTF12_BAC','DFT_QCI_thermo','CHN','GRI-Mech3.0-N'],
    reactionLibraries = [],
    seedMechanisms = [],
    kineticsDepositories = 'default', 
#   kineticsFamilies = ['Intra_R_Add_Exocyclic'],  # Select a few families
    kineticsFamilies = 'all',   # select 'all' or 'default' for the families
    kineticsEstimator = 'rate rules',
)

options(
    verboseComments=True,  # Set to True for detailed kinetics comments
)
"""

##### List all species you want reactions between

In [48]:
speciesList = """
species(
    label = "O",
    structure = SMILES("[O]"))
    
species(
    label = "NNH",
    structure = SMILES("N=[N]"))
"""

In [49]:
# Write input file to disk
inputFile = open('temp/input.py','w')
inputFile.write(database)
inputFile.write(speciesList)
inputFile.close()

In [50]:
# Execute generate reactions
from rmgpy.tools.generate_reactions import *
rmg = RMG(inputFile='temp/input.py', outputDirectory='temp')
rmg = execute(rmg)

In [51]:
# Pick some temperature to evaluate the forward and reverse kinetics
T = 623.0 # K

In [52]:
if len(rmg.reactionModel.outputReactionList) == 0:
    print 'No reactions generated'
for rxn in rmg.reactionModel.outputReactionList:
    print '========================='
    display(rxn)
    print 'Reaction Family = {0}'.format(rxn.family)
    print ''
    print 'Reactants'
    for reactant in rxn.reactants:
        print 'Label: {0}'.format(reactant.label)
        print 'SMILES: {0}'.format(reactant.molecule[0].toSMILES())
        print ''
    print 'Products'
    for product in rxn.products:
        print 'Label: {0}'.format(product.label)
        print 'SMILES: {0}'.format(product.molecule[0].toSMILES())
    print ''
    print rxn.toChemkin()
    print ''
    print 'Heat of Reaction = {0:.2F} kcal/mol'.format(rxn.getEnthalpyOfReaction(623.0)/4184)
    print 'Forward kinetics at {0} K: {1:.2E}'.format(T, rxn.getRateCoefficient(T))

    reverseRate = rxn.generateReverseRateCoefficient()
    print 'Reverse kinetics at {0} K: {1:.2E}'.format(T, reverseRate.getRateCoefficient(T))

Reaction Family = R_Addition_MultipleBond

Reactants
Label: N2
SMILES: N#N

Label: [H]
SMILES: [H]

Products
Label: NNH
SMILES: N=[N]

! Template reaction: R_Addition_MultipleBond
! Flux pairs: N2, NNH(2); H(3), NNH(2); 
! Average of [Average of [Average of [N3t_Ct-H;HJ]] + Average of [Average of [N3t_N3t;CH2_triplet] + Average of [N3t_N3t;CH_quartet]]]
! Estimated using average of templates [N3t_R;HJ] + [N3t_N3t;YJ] for rate rule [N3t_N3t;HJ]
! Multiplied by reaction path degeneracy 2
N2+H(3)=NNH(2)                                      2.636e+30 -6.662    9.378    


Heat of Reaction = 6.53 kcal/mol
Forward kinetics at 623.0 K: 3.26E+02
Reverse kinetics at 623.0 K: 7.50E+10


Reaction Family = R_Recombination

Reactants
Label: [H]
SMILES: [H]

Label: [N]=[N]
SMILES: [N]=[N]

Products
Label: NNH
SMILES: N=[N]

! Template reaction: R_Recombination
! Flux pairs: H(3), NNH(2); N2(4), NNH(2); 
! Average of [Average of [Average of [Average of [Average of [N3d-OdOs;O_rad] + Average of [Average of [O_pri_rad;N3d-OdOs] + O_rad;N3d-OdOs]] + Average
! of [Average of [Average of [O_pri_rad;N3d-OdOs]] + Average of [Average of [O_pri_rad;N3d-OdOs] + O_rad;N3d-OdOs]]] + Average of [Average of [Average
! of [Average of [O_pri_rad;N3d-OdOs]]] + Average of [Average of [Average of [O_pri_rad;N3d-OdOs]] + Average of [Average of [O_pri_rad;N3d-OdOs] + O_rad
! ;N3d-OdOs]]]] + Average of [H_rad;H_rad + Average of [Average of [H_rad;SsJ-H]] + Average of [H_rad;Ct_rad/Ct] + Average of [H_rad;O_pri_rad] +
! H_rad;O2_birad + Average of [H_rad;Cd_pri_rad + H_rad;Cd_allenic] + H_rad;Cb_rad + Average of [H_rad;CO_pri_rad] + H_rad;Cs_rad]]
! Estimated using average of templates [Y_rad;N3

Reaction Family = Disproportionation

Reactants
Label: O
SMILES: [O]

Label: NNH
SMILES: N=[N]

Products
Label: N2
SMILES: N#N
Label: [OH]
SMILES: [OH]

! Template reaction: Disproportionation
! Flux pairs: O(1), HO(5); NNH(2), N2; 
! Exact match found for rate rule [O_atom_triplet;N3d/H_d_Nrad]
O(1)+NNH(2)=N2+HO(5)                                1.700e+08 1.500     -0.890   


Heat of Reaction = -110.25 kcal/mol
Forward kinetics at 623.0 K: 5.42E+06
Reverse kinetics at 623.0 K: 3.46E-32


Reaction Family = H_Abstraction

Reactants
Label: O
SMILES: [O]

Label: NNH
SMILES: N=[N]

Products
Label: [N]=[N]
SMILES: [N]=[N]
Label: [OH]
SMILES: [OH]

! Template reaction: H_Abstraction
! Flux pairs: O(1), HO(5); NNH(2), N2(4); 
! Average of [N3s_H;O_atom_triplet + Average of [Average of [N3d/H/NonDeC;O_atom_triplet + N3d/H/NonDeO;O_atom_triplet + N3d/H/NonDeN;O_atom_triplet] +
! Average of [N3d/H/OneDeCO;O_atom_triplet]]]
! Estimated using an average for rate rule [N3_H;O_atom_triplet]
! Kinetics were estimated in this direction instead of the reverse because:
! Both directions are estimates, but this direction is exergonic.
! dHrxn(298 K) = -162.00 kJ/mol, dGrxn(298 K) = -170.60 kJ/mol
O(1)+NNH(2)=N2(4)+HO(5)                             1.273e+08 1.545     2.466    


Heat of Reaction = -38.29 kcal/mol
Forward kinetics at 623.0 K: 3.61E+05
Reverse kinetics at 623.0 K: 2.45E-10


Reaction Family = Oa_R_Recombination

Reactants
Label: O
SMILES: [O]

Label: NNH
SMILES: N=[N]

Products
Label: N=N[O]
SMILES: [NH]N=O

! Template reaction: Oa_R_Recombination
! Flux pairs: O(1), HN2O(6); NNH(2), HN2O(6); 
! Exact match found for rate rule [Y_rad;Oa]
O(1)+NNH(2)=HN2O(6)                                 1.000e+13 0.000     0.000    


Heat of Reaction = -70.93 kcal/mol
Forward kinetics at 623.0 K: 1.00E+07
Reverse kinetics at 623.0 K: 8.31E-10


Reaction Family = Disproportionation

Reactants
Label: NNH
SMILES: N=[N]

Label: NNH
SMILES: N=[N]

Products
Label: N2
SMILES: N#N
Label: N=N
SMILES: N=N

! Template reaction: Disproportionation
! Flux pairs: NNH(2), H2N2(7); NNH(2), N2; 
! Average of [N3d_rad/O;N3d/H_d_Nrad]
! Estimated using template [N3d_rad;N3d/H_d_Nrad] for rate rule [N3d_rad/N;N3d/H_d_Nrad]
NNH(2)+NNH(2)=N2+H2N2(7)                            1.200e+06 2.000     -1.190   


Heat of Reaction = -71.96 kcal/mol
Forward kinetics at 623.0 K: 1.22E+06
Reverse kinetics at 623.0 K: 1.14E-17


Reaction Family = Disproportionation

Reactants
Label: [N]=[N]
SMILES: [N]=[N]

Label: N[N]
SMILES: N[N]

Products
Label: NNH
SMILES: N=[N]
Label: NNH
SMILES: N=[N]

! Template reaction: Disproportionation
! Flux pairs: N2(4), NNH(2); H2N2(8), NNH(2); 
! Average of [Average of [NH2_rad;N3s/H2_s_Nbirad]]
! Estimated using template [N3_rad;N3s/H2_s_Nbirad] for rate rule [N3d_rad;N3s/H2_s_Nbirad]
! Multiplied by reaction path degeneracy 4
N2(4)+H2N2(8)=NNH(2)+NNH(2)                         7.200e+06 1.940     -1.150   


Heat of Reaction = -56.84 kcal/mol
Forward kinetics at 623.0 K: 4.81E+06
Reverse kinetics at 623.0 K: 4.55E-13


Reaction Family = H_Abstraction

Reactants
Label: [N]=[N]
SMILES: [N]=[N]

Label: N=N
SMILES: N=N

Products
Label: NNH
SMILES: N=[N]
Label: NNH
SMILES: N=[N]

! Template reaction: H_Abstraction
! Flux pairs: N2(4), NNH(2); H2N2(7), NNH(2); 
! Average of [Average of [N3d/H/NonDeN;NH2_rad]]
! Estimated using an average for rate rule [N3d/H/NonDeN;N3_rad]
! Multiplied by reaction path degeneracy 4
! Kinetics were estimated in this direction instead of the reverse because:
! Both directions are estimates, but this direction is exergonic.
! dHrxn(298 K) = -0.00 kJ/mol, dGrxn(298 K) = -0.00 kJ/mol
! Ea raised from -4.8 to 0.0 kJ/mol to match endothermicity of reaction.
N2(4)+H2N2(7)=NNH(2)+NNH(2)                         7.200e+06 1.940     0.002    


Heat of Reaction = -0.00 kcal/mol
Forward kinetics at 623.0 K: 1.90E+06
Reverse kinetics at 623.0 K: 1.90E+06


Reaction Family = R_Recombination

Reactants
Label: NNH
SMILES: N=[N]

Label: NNH
SMILES: N=[N]

Products
Label: N(=N)N=N
SMILES: N(=N)N=N

! Template reaction: R_Recombination
! Flux pairs: NNH(2), H2N4(9); NNH(2), H2N4(9); 
! Average of [Average of [Average of [Average of [N3d-OdOs;O_rad] + Average of [Average of [O_pri_rad;N3d-OdOs] + O_rad;N3d-OdOs]] + Average of [Average
! of [Average of [O_pri_rad;N3d-OdOs]] + Average of [Average of [O_pri_rad;N3d-OdOs] + O_rad;N3d-OdOs]]] + Average of [Average of [Average of [N3d-
! OdOs;O_rad] + Average of [Average of [O_pri_rad;N3d-OdOs] + O_rad;N3d-OdOs]] + Average of [Average of [Average of [O_pri_rad;N3d-OdOs]] + Average of
! [Average of [O_pri_rad;N3d-OdOs] + O_rad;N3d-OdOs]]]]
! Estimated using average of templates [Y_rad;N3d_rad] + [N3d_rad;Y_rad] for rate rule [N3d-N3d;N3d-N3d]
NNH(2)+NNH(2)=H2N4(9)                               1.025e+13 0.120     0.000    


Heat of Reaction = -57.31 kcal/mol
Forward kinetics at 623.0 K: 2.22E+07
Reve